In [1]:
import time
import requests
import numpy as np
import pandas as pd
from web3 import Web3
from tqdm.auto import tqdm
from pandarallel import pandarallel 
pandarallel.initialize(progress_bar=True, nb_workers=16)

def timestamp_to_date(timestamp):
    import datetime
    
    def days_ago(date_str):
        """Returns the number of days ago a date (in the format 'YYYY-MM-DD') was from the current date."""
        date = datetime.datetime.strptime(date_str, "%Y-%m-%d")
        today = datetime.datetime.today()
        days_ago = (today - date).days
        return f"{days_ago} days ago"
        
    """Converts a Unix timestamp (in seconds) to a human-readable date in 12-hour format."""
    date = datetime.datetime.fromtimestamp(timestamp)
    human_date = date.strftime("%Y-%m-%d")
    human_time = date.strftime("%I:%M:%S %p")
    return f"{days_ago(human_date)} {human_time}"

def add_address(x):
    from web3 import Web3
    w3 = Web3(Web3.HTTPProvider('https://eth-mainnet.gateway.pokt.network/v1/5f3453978e354ab992c4da79'))
    return w3.eth.getTransactionReceipt(x)['from'].lower()

def profit_calculator(wallet_address):
    wallet = df_new[df_new["Wallet Address"] == wallet_address]
    buy = wallet[wallet["Action"] == 'Buy']["Amount (Out)"].apply(lambda x: float(x)).sum()
    sell = wallet[wallet["Action"] == 'Sell']["Amount (In)"].apply(lambda x: float(x)).sum()
    profit = lamba_x(sell-buy) if sell > 0 else 0
    multiplier = ((sell-buy) / buy) if buy != 0 else 0
    data = {"Address": wallet_address, "Buy": lamba_x(buy), "Sell": lamba_x(sell), "Profit": profit, "Multiplier": multiplier}
    return data

lamba_x = lambda x: float(f"{x:.2f}")

# Define the Ethereum node endpoint
# TODO: download https://etherscan.io/exportData?type=dextracker&q=contractAddress replace contractAddress with erc20 token then rename it to trades.csv
df = pd.read_csv('trades.csv')

tqdm.pandas()
df['Wallet Address'] = df['Txn Hash'].parallel_apply(add_address)
df_new = df.copy()
df_new['Date Time'] = df_new['UnixTimestamp'].apply(timestamp_to_date)
df_new["Amount (In)"] = df_new["Amount (In)"].apply(lamba_x)
df_new["Amount (Out)"] = df_new["Amount (Out)"].apply(lamba_x)
df_new = df_new[["Date Time","Wallet Address","Action","Amount (Out)","Token (Out)","Amount (In)","Token (In)","Txn Hash"]]

arr = list(df_new['Wallet Address'].unique())
wallet_dict = []
for address in arr:
    data = profit_calculator(address)
    wallet_dict.append(data)

df_final = pd.DataFrame(wallet_dict)
gem_wallets = df_final[(df_final["Buy"] > 0.01) & (df_final["Multiplier"] >= 5)].sort_values(by=['Multiplier'], ascending=False)
gem_wallets.to_csv("gem_wallets.csv", index=False)
gem_wallets.head()

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


,Address,Buy,Sell,Profit,Multiplier
556,0x713e5145c3152201a54d1a1aabc9e5b482063515,0.02,0.90,0.88,44.0
553,0xf3e5f42bbc3e8707f612e40e9beec9ab407ca335,0.02,0.86,0.84,42.0
606,0x4cd3e5e1fb4ffba3c11c919dc475585ed0534875,0.02,0.83,0.81,40.5
564,0xd8657007a67e4e8194241e04e574b3337bda70c8,0.02,0.72,0.70,35.0
690,0x1c134fde47d09f78dfe8f61a22c03cb2da68e904,0.03,0.90,0.87,29.0
